## Problem 3
https://projecteuler.net/problem=3

In [1]:
using Primes, BenchmarkTools

#### Solution 1

In [2]:
@btime maximum(keys(Primes.factor(600851475143)))

  1.637 μs (1 allocation: 128 bytes)


6857

#### Solution 2

In [3]:
function next_prime(p::Int64)::Int64
    p += 1
    while !isprime(p)
        p += 1
    end
    return p
end;

In [4]:
function prime_factorization(n::Int64)::Vector{Int64}
    factors = Int64[]
    index = 0
    prime = 0
    while n != 1
        index += 1
        prime = next_prime(prime)
        while n % prime == 0
            push!(factors, prime)
            n = Int64(n / prime)
        end
    end
    return factors 
end

prime_factorization (generic function with 1 method)

In [5]:
@assert maximum(prime_factorization(13195)) == 29

In [6]:
@btime maximum(prime_factorization(600851475143))

  20.459 μs (2 allocations: 144 bytes)


6857